In [2]:
import pydantic


ImportError: Numba needs NumPy 1.21 or less

In [2]:
import os
import re
from joblib import Parallel, delayed
from sim.util import bind_results
import pickle as pkl
import numpy as np
import pandas as pd
from sim import load_inputs
from sim.dy.model_cascade import ModelCascade
import sim.dy.keys as I

In [3]:
def fn_det(y0, p, m, intv, dt):
    _, ms, _ = m.simulate_onward(y0=y0, p=p, intv=intv, dt = dt)

    return {
        'DetACF': (ms.N_ACF_Detected * dt).sum(),
        'DetPCF': (ms.N_Pub_Detected * dt).sum() + (ms.N_Pri_Detected * dt).sum(),
        'DurUT': (ms.N_UT * dt).sum(),
        'ACF_Reached': ms.N_ACF_Reached.sum() * dt
    }

    return ms

inputs = load_inputs('data/pars.json')
m = ModelCascade(inputs, year0=1970, year=2022)
dt = 0.01

In [4]:
p_sp0 = 0.6

y0 = np.zeros((I.N_State_TB, I.N_State_Strata))
y0[I.Asym_Sn_DS, I.RiskHi] = (1 - p_sp0)
y0[I.Asym_Sp_DS, I.RiskHi] = p_sp0
y0 = y0.reshape(-1)


In [9]:
ds = [d for d in os.listdir("out") if d.startswith('dy_')]
ds = ['dy_5%_2', 'dy_20%_5']


In [11]:
rates = np.linspace(0, 15, 31)

In [12]:
for path in ds:
    print(path)
    out_path = f'out/{path}'

    # Posterior run
    post = pkl.load(open(f'{out_path}/y0_national.pkl', 'rb'))
    
    mss = list()
    for rate in rates:


        intv_m = {'ACFPlain': {'R_ACF': rate, 'Type': 'mod'}}
        intv_h = {'ACFPlain': {'R_ACF': rate, 'Type': 'high'}}

        with Parallel(n_jobs=3, verbose=8) as parallel:
            mss0 = parallel(delayed(fn_det)(y0, po['pars'], m, intv_m, dt) for po in post[:20])

        mss0 = pd.DataFrame(mss0).assign(R_ACF=rate, ACF="Mod")
        mss.append(mss0)


        with Parallel(n_jobs=3, verbose=8) as parallel:
            mss0 = parallel(delayed(fn_det)(y0, po['pars'], m, intv_h, dt) for po in post[:20])

        mss0 = pd.DataFrame(mss0).assign(R_ACF=rate, ACF="High")
        mss.append(mss0)
    mss = pd.concat(mss)
    mss.to_csv(f'{out_path}/Runs_CDR.csv')

dy_5%_2


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    5.2s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    6.7s remaining:    0.7s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    7.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.1s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.7s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.1s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.7s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.1s
[Paral

[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.2s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.8s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.2s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.3s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.9s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent worker

[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.1s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.2s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.6s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.4s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.6s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.4s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.2s f

dy_20%_5


[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    2.9s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.4s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.0s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.5s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.0s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.7s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.2s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    4.7s remaining:    0.4s
[

[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.7s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.5s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.2s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.1s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    5.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.3s remaining:    0.5s
[

[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.6s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.6s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.8s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.5s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.4s remaining:    0.5s
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:    6.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.5s
[Parallel(n_jobs=3)]: Done  18 out of  20 | elapsed:    5.3s remaining:    0.5s
[